In [1]:
# code to manually check performance of a model on validation set
# !!! skeleton code
import model
import pan_loader
import base_config
import loss_functions as L


import visualize
import utils

import torch
import numpy as np

import os
import time
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
data_dir = "/home/aravind/dataset/"
ann_dir = data_dir + "annotations/panoptic/"

train_img_dir = data_dir + "train2017/"
train_seg_dir = ann_dir + "panoptic_train2017/"
train_ann_json = ann_dir + "panoptic_train2017.json"

val_img_dir = data_dir + "val2017/"
val_seg_dir = ann_dir + "panoptic_val2017/"
val_ann_json = ann_dir + "panoptic_val2017.json"

train_img_dir = val_img_dir 
train_seg_dir = val_seg_dir 
train_ann_json = val_ann_json 

In [3]:
with open(val_ann_json,"r") as f:
    val_ann = json.load(f)
with open(train_ann_json,"r") as f:
    train_ann = json.load(f)

In [4]:
class InferenceConfig(base_config.Config):
    IMAGES_PER_GPU = 1
config = InferenceConfig()

In [5]:
train_loader = pan_loader.get_loader(train_img_dir, train_seg_dir, train_ann, config)
val_loader = pan_loader.get_loader(val_img_dir, val_seg_dir, val_ann, config)

In [6]:
model_dir = "models/"
model_name = "lol_0.pt"

In [8]:
net = model.hgmodel()

pretrained_dict = torch.load(model_dir+model_name)
net_dict = net.state_dict()

pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}
net_dict.update(pretrained_dict) 
net.load_state_dict(net_dict)

net.eval()
net = net.cuda()

In [ ]:
from PIL import Image
for i, data in enumerate(val_loader,0):
    print("Image:\t%d"%i)
    images, impulses, instance_masks, cat_ids = utils.cudify_data(data)
    
    impulses.unsqueeze_(1); instance_masks.unsqueeze_(1)
    with torch.no_grad():
        outs = net([images,impulses])
    pred_masks, pred_class = outs
    pred_masks = pred_masks.squeeze().sigmoid().cpu()
    pred_masks = (pred_masks > 0.5)
    pred_labels = pred_class.argmax(1).cpu()
    
    images, impulses, instance_masks, cat_ids = images.cpu(), impulses.cpu(), instance_masks.cpu(), cat_ids.cpu()

    impulses.squeeze(1); instance_masks.squeeze(1)
    visualize.visualize_targets([images, impulses, instance_masks, cat_ids], config, "in")
    visualize.visualize_targets([images, impulses, pred_masks, pred_labels], config, "out")
    input()

Image:	0
(8, 3, 224, 224) (8, 1, 224, 224)
(8, 3, 224, 224) (8, 1, 224, 224)

Image:	1
(5, 3, 224, 224) (5, 1, 224, 224)
(5, 3, 224, 224) (5, 1, 224, 224)

Image:	2
(4, 3, 224, 224) (4, 1, 224, 224)
(4, 3, 224, 224) (4, 1, 224, 224)
